# 1. Import and Install Dependencies

In [1]:
pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model (Make detections)
mp_drawing = mp.solutions.drawing_utils # Drawing utilities (Draw detections)

In [4]:
# the feed received from open cv is in bgr format, convert it to make it usable
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
#  visualizing landmark to the frame
# take image and results from media pipe as input
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [8]:
import pandas as pd
import json

# Parse the JSON data into a Python object
WLASL = pd.read_csv("WLASL_v0.3_raw.csv") # 21000 records
WLASL['gloss'] = WLASL['gloss'].fillna(method='ffill')
WLASL.head(10)
WLASL['video_id']=WLASL['video_id'].astype('int')

In [9]:
WLASL.dtypes

gloss           object
bbox             int64
fps              int64
frame_end        int64
frame_start      int64
instance_id      int64
signer_id        int64
source          object
split           object
url             object
variation_id     int64
video_id         int32
dtype: object

In [10]:
WLASL[WLASL['video_id']==412]
# WLASL['video_id'].unique()

,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id
3361,about,0,25,240,184,0,9,asl5200,val,https://www.youtube.com/watch?v=jPn0FQxkdBc,0,412


In [11]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data - WLASL') 

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou'])
import numpy as np

In [12]:
WLASL.shape

(21083, 12)

In [13]:
DATA_PATH

'MP_Data - WLASL'

In [14]:
import glob
path = 'WLASL Data'

mp4_files = glob.glob(path + "/*")

videos_wlasl=[int(i[len(i)-9:len(i)-4]) for i in mp4_files]
files=[i for i in mp4_files]

fair_videos=pd.DataFrame(data=videos_wlasl,columns=['video_id'])
fair_videos['folder_loc']=files # transformed

In [15]:
fair_asl=pd.merge(WLASL,fair_videos,how='inner',on='video_id')
actions=np.array(fair_asl['gloss'])

In [16]:
fair_asl.head(5)

,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,folder_loc
0,book,385,25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,WLASL Data\69241.mp4
1,book,462,25,-1,1,10,31,signschool,train,https://signstock.blob.core.windows.net/signsc...,0,7069,WLASL Data\07069.mp4
2,book,234,25,-1,1,17,36,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,0,7068,WLASL Data\07068.mp4
3,book,131,25,-1,1,22,59,asldeafined,train,https://media.asldeafined.com/vocabulary/14666...,0,7070,WLASL Data\07070.mp4
4,book,162,25,-1,1,24,12,aslsearch,val,http://www.aslsearch.com/signs/videos/book.mp4,0,7099,WLASL Data\07099.mp4


In [17]:
len(actions)

1704

# FAIR VIDEOS

In [18]:
sequences=16 # To find annotation that has the maximum sequence length

In [20]:
# Create folders for all gthe labels(RUN ONLY ONCE)
for action in actions: 
    for sequence in range(1,sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [19]:
# fair_asl.to_csv('Data/fair_videos.csv')
fair_asl.head(2)

,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,folder_loc
0,book,385,25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,WLASL Data\69241.mp4
1,book,462,25,-1,1,10,31,signschool,train,https://signstock.blob.core.windows.net/signsc...,0,7069,WLASL Data\07069.mp4


In [73]:
# fair_asl['folder_loc'][0]

# 5. Collect Keypoint Values for Training and Testing

DONT NEED TO RUN AFTER GETTING THE.NPY FILES ONCE

In [ ]:
# Load video
for j in range(0,len(fair_asl)):
    sequence=1
    # try:
    while len(os.listdir(os.path.join(DATA_PATH,fair_asl['gloss'][j],str(sequence))))!=0:
        sequence+=1
    cap = cv2.VideoCapture(fair_asl['folder_loc'][j])
    i=1
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # Loop through frames
        while True:
            # Read frame
            ret, frame = cap.read()
            if not ret:
                break 
            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, fair_asl['gloss'][j],str(sequence), str(i))
            np.save(npy_path, keypoints)
            i+=1
        cap.release()

    # except:
    #     pass

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Removing all the empty folders
for i in os.listdir(DATA_PATH):
    for j in os.listdir(os.path.join(DATA_PATH,i)):
        if len(os.listdir(os.path.join(DATA_PATH,i,j)))==0:
            os.rmdir(os.path.join(DATA_PATH,i,j))
        else:
            continue

In [20]:
# Finding max length
length=[]
for i in os.listdir(DATA_PATH):
    for j in os.listdir(os.path.join(DATA_PATH,i)):
        length.append(len(os.listdir(os.path.join(DATA_PATH,i,j))))

In [21]:
# finding max length from all videos converted to frame
max(length)

195

In [22]:
# Verifying # of videos
videos=0
for i in os.listdir(DATA_PATH):
    for j in os.listdir(os.path.join(DATA_PATH,i)):
        videos+=1

In [23]:
videos

1704

# 6. Preprocess Data and Create Labels and Features

In [24]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [25]:
actions=np.array(list(set(actions)))
label_map = {label:num for num, label in enumerate(actions)}

In [26]:
# annotations
len(label_map)

186

In [27]:
actions.shape

(186,)

In [28]:
import numpy as np

In [29]:
res = np.load("MP_Data - WLASL/ball/1/1.npy")

In [30]:
res

array([ 0.51163751,  0.2182218 , -0.84421432, ...,  0.        ,
        0.        ,  0.        ])

In [31]:
res.shape

(1662,)

In [32]:
res1 = np.load("MP_Data - WLASL/book/5/4.npy")

In [33]:
res1

array([ 0.47027978,  0.31408572, -0.75616115, ...,  0.        ,
        0.        ,  0.        ])

In [34]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequence_length = max(length)
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(1, sequence_length+1):
            try:
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            except:
                continue
        sequences.append(window)
        labels.append(label_map[action])


# Pad sequences to a fixed length
padded_sequences = pad_sequences(sequences, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')

In [35]:
len(window)

56

In [36]:
len(sequences)

1704

In [43]:
len(labels)

1704

In [44]:
np.array(padded_sequences).shape

(1704, 195, 1662)

In [45]:
np.array(labels).shape

(1704,)

In [46]:
X = np.array(padded_sequences)

In [47]:
X.shape

(1704, 195, 1662)

In [48]:
y = to_categorical(labels).astype(int)

In [49]:
y.shape

(1704, 186)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [51]:
X_train.shape

(1363, 195, 1662)

In [52]:
X_test.shape

(341, 195, 1662)

# 7. Build and Train LSTM Neural Network

In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [54]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [55]:
X_train.shape,X_test.shape

((1363, 195, 1662), (341, 195, 1662))

In [50]:
y_train.shape,y_test.shape

((1363, 186), (341, 186))

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(SimpleRNN(units=256, activation='tanh', return_sequences=True, input_shape=(195, 1662)))
model.add(Dropout(0.5))
model.add(SimpleRNN(units=64, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=len(actions), activation='softmax'))

optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=2000, batch_size=128,validation_data=(X_test,y_test))

Epoch 1/2000
11/11 [==============================] - 8s 656ms/step - loss: 5.2854 - accuracy: 0.0066 - val_loss: 5.2437 - val_accuracy: 0.0029
Epoch 2/2000
11/11 [==============================] - 7s 624ms/step - loss: 5.2717 - accuracy: 0.0029 - val_loss: 5.2514 - val_accuracy: 0.0059
Epoch 3/2000
11/11 [==============================] - 7s 625ms/step - loss: 5.2417 - accuracy: 0.0051 - val_loss: 5.2417 - val_accuracy: 0.0088
Epoch 4/2000
11/11 [==============================] - 7s 637ms/step - loss: 5.2351 - accuracy: 0.0059 - val_loss: 5.2435 - val_accuracy: 0.0029
Epoch 5/2000
11/11 [==============================] - 7s 628ms/step - loss: 5.2183 - accuracy: 0.0095 - val_loss: 5.2383 - val_accuracy: 0.0029
Epoch 6/2000
11/11 [==============================] - 7s 668ms/step - loss: 5.2278 - accuracy: 0.0037 - val_loss: 5.2507 - val_accuracy: 0.0059
Epoch 7/2000
11/11 [==============================] - 8s 782ms/step - loss: 5.2242 - accuracy: 0.0110 - val_loss: 5.2473 - val_accuracy:

# 8. Make Predictions

In [52]:
res = model.predict(X_test)

11/11 [==============================] - 4s 356ms/step


In [53]:
actions[np.argmax(res[3])]

'run'

In [58]:
actions[np.argmax(y_test[15])]

'far'

# 9. Save Weights

In [58]:
model.save('action.h5')

In [ ]:
del model

In [59]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [60]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [62]:
yhat = model.predict(X_test)

11/11 [==============================] - 4s 343ms/step


In [63]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [131]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[337,   1],
        [  3,   0]],

       [[338,   1],
        [  2,   0]],

       [[333,   6],
        [  2,   0]],

       [[335,   4],
        [  2,   0]],

       [[339,   0],
        [  2,   0]],

       [[338,   2],
        [  1,   0]],

       [[339,   0],
        [  2,   0]],

       [[338,   1],
        [  2,   0]],

       [[336,   5],
        [  0,   0]],

       [[340,   1],
        [  0,   0]],

       [[336,   1],
        [  4,   0]],

       [[340,   1],
        [  0,   0]],

       [[340,   1],
        [  0,   0]],

       [[340,   0],
        [  1,   0]],

       [[336,   3],
        [  2,   0]],

       [[338,   2],
        [  1,   0]],

       [[338,   3],
        [  0,   0]],

       [[339,   1],
        [  1,   0]],

       [[339,   1],
        [  1,   0]],

       [[339,   0],
        [  2,   0]],

       [[339,   0],
        [  1,   1]],

       [[336,   4],
        [  1,   0]],

       [[337,   1],
        [  3,   0]],

       [[336,   0],
        [  5, 

In [64]:
accuracy_score(ytrue, yhat)

0.01466275659824047

# 11. Test in Real Time

In [119]:
from scipy import stats

In [120]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [80]:
#TRYING Capture video from the camera
cap = cv2.VideoCapture(0)

# Loop over the frames from the video stream
while True:
    # Capture the next frame
    ret, frame = cap.read()

    # Resize the frame to the required size
    resized_frame = cv2.resize(frame, (224, 224))

    # Normalize the pixel values
    normalized_frame = resized_frame / 255.0

    # Add a batch dimension
    batched_frame = np.expand_dims(normalized_frame, axis=0)

    # Get the predictions from the model
    predictions = model.predict(batched_frame)

    # Get the index of the action with the highest probability
    action_index = np.argmax(predictions)

    # Get the probability distribution
    distribution = tf.nn.softmax(predictions[0])

    # Get the probabilities for each action
    probabilities = distribution[action_index].numpy()

    # Show the probabilities on the frame
    image = prob_viz(probabilities, actions, resized_frame, colors)
    cv2.imshow('Gesture recognition', image)

    # Check if the user has pressed the "q" key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/simple_rnn_2/transpose' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._

In [78]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1800x1800 with 0 Axes>

In [122]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.1

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        if ret==True:
            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
#             sequence.append(keypoints)
            sequence.insert(0,keypoints)
            sequence = sequence[:30]

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))


            #3. Viz logic
            if res[np.argmax(res)]>threshold:
                if len(sentence)>0:
                    if actions[np.argmax(res)]!=sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
#             if np.unique(predictions[-10:])[0]==np.argmax(res): 
#                 if res[np.argmax(res)] > threshold: 

#                     if len(sentence) > 0: 
#                         if actions[np.argmax(res)] != sentence[-1]:
#                             sentence.append(actions[np.argmax(res)])
#                     else:
#                         sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


IndexError: index 14114 is out of bounds for axis 0 with size 341